In [2]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [3]:
park = Customer('Park', 100)
park

Customer(name='Park', fidelity=100)

In [4]:
class LineItem:
  def __init__(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

In [5]:
class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '_total'):
      self._total = sum(item.total() for item in self.cart)
    return self._total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion.discount(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

In [6]:
class Promotion(ABC):
  @abstractmethod
  def discount(self, order):
    pass

In [7]:
class FidelitypPromo(Promotion):
  def discount(self, order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
  def discount(self, order):
    discount = 0
    for item in order.cart:
      if item.quantity >= 20:
        discount += item.total() * 0.1
    return discount

class LargeOrderPromo(Promotion):
  def discount(self, order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * 0.07
    return 0

In [8]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [9]:
Order(joe, cart, FidelitypPromo())

<Order total: 42.00 due: 42.00>

In [10]:
Order(ann, cart, FidelitypPromo())

<Order total: 42.00 due: 39.90>

In [11]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

In [12]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [13]:
long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]

In [14]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [15]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [16]:
class Order:
  def __init__(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '_total'):
      self._total = sum(item.total() for item in self.cart)
    return self._total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion(self)
    return self.total() - discount

  def __repr__(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

In [18]:
def fidelity_promo(order):
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

def large_order_promo(order):
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * 0.07
  return 0

In [19]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [20]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [21]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [22]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [23]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  return max(promo(order) for promo in promos)

In [24]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [25]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [26]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [27]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', '_i2', 'abstractmethod', 'namedtuple', 'Customer', '_i3', 'park', '_3', '_i4', 'LineItem', '_i5', 'Order', '_i6', 'Promotion', '_i7', 'FidelitypPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i8', 'joe', 'ann', 'cart', '_i9', '_9', '_i10', '_10', '_i11', 'banana_cart', '_i12', '_12', '_i13', 'long_order', '_i14', '_14', '_i15', '_15', '_i16', '_i17', 'fidelity_promo', '_i18', 'bulk_item_promo', 'large_order_promo', '_i19', '_19', '_i20', '_20', '_i21', '_21', '_i22', '_22', '_i23', 'promos', 'best_promo', '_i24', '_24', '_i25', '_25', '_i26', '_26', '_i27'])

In [28]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [29]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']

promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]